In [1]:
rm(list=ls())
gc()
setwd("/scratch/AG_Ohler/CheWei/proj_sc/cbpsc")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,574700,30.7,1245252,66.6,853948,45.7
Vcells,1076007,8.3,8388608,64.0,1820945,13.9


In [2]:
as.numeric(system("awk '/MemFree/ {print $2}' /proc/meminfo", intern=TRUE))
options(future.globals.maxSize = 600000 * 1024^2)

[1] 240912980

In [3]:
suppressMessages(library(Seurat))
suppressMessages(library(ggplot2))

In [4]:
suppressMessages(library(dplyr))

In [5]:
sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /fast/home/c/chsu/anaconda3/envs/tradeseq/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.utf-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.utf-8        LC_COLLATE=en_US.utf-8    
 [5] LC_MONETARY=en_US.utf-8    LC_MESSAGES=en_US.utf-8   
 [7] LC_PAPER=en_US.utf-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.utf-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_1.0.7        ggplot2_3.3.5      SeuratObject_4.0.4 Seurat_4.1.0      

loaded via a namespace (and not attached):
  [1] Rtsne_0.15            colorspace_2.0-3      deldir_1.0-6         
  [4] ellipsis_0.3.2        ggridges_0.5.3        IRdisplay_1.1        
  [7] RcppHNSW_0.3.0       

## Load downloaded "Root_Atlas_seu4.rds" from GEO:GSE152766 

In [6]:
rc.integrated <- readRDS('../../kallisto-bus/output/Integration/Root_Atlas_seu4.rds')

In [ ]:
# Need to change the directory accordingly 
label_transfer_init <- function(sample.name){
    seu <-read_seu(dir = paste0("../../kallisto-bus/output/COPILOT_RDS_LENIENT_DR/",sample.name,"_COPILOT.rds"))
    lt.anchors <- FindTransferAnchors(reference = rc.integrated, query = seu, normalization.method = "SCT", npcs = 50 ,dims = 1:50)
    ## time.celltype.anno.Li is the dev stage reannotation
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.celltype.anno.Li, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.celltype.anno.Li <- seu@meta.data$predicted.id
    ## time.celltype.anno.Li.crude is the dev stage reannotation without sub cell types
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.celltype.anno.Li.crude, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.celltype.anno.Li.crude <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.celltype.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.celltype.anno <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.celltype.anno.crude, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.celltype.anno.crude <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$celltype.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$celltype.anno <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$celltype.anno.crude, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$celltype.anno.crude <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.anno <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$branch.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$branch.anno <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$consensus.time.group, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$consensus.time.group <- seu@meta.data$predicted.id
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$consensus.time.group.50, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$consensus.time.group.50 <- seu@meta.data$predicted.id
    saveRDS(seu, file = paste0("../../kallisto-bus/output/COPILOT_RDS_LENIENT_DR/",sample.name,"_COPILOT_seu4.rds"))
}

In [13]:
sample.list <- c("dc1", "dc2", "sc_1", "sc_12", "sc_2", "sc_43", "sc_44", "sc_45", "sc_46", "sc_47", "sc_48", "sc_49", "sc_5", "sc_50", 
                 "sc_111", "sc_112", "sc_113", "sc_114", "sc_122", "sc_123", "sc_124", "sc_125", "sc_126", "sc_127", "sc_128", "sc_129",
                 "sc_130", "sc_131", "sc_132", "sc_133", "sc_134", "sc_135", "sc_136", "sc_137", "sc_147", "sc_148", "sc_149", "sc_150", "sc_151", "sc_152", "sc_153", "sc_154",
                "pp1", "col0", "tnw1", "tnw2", "sc_9_at", "sc_10_at", "sc_11", "sc_30", "sc_31", "sc_37", "sc_40", "sc_51")

In [16]:
for (i in 1:length(sample.list)){
 label_transfer_init(sample.list[i])
}

Normalizing query using reference SCT model

Performing PCA on the provided reference using 1844 features as input.

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 21082 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Finding integration vectors

Finding integration vector weights

Predicting cell 